## Visualize after filtering

In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter

# --------------------------------------------------------------
# 基本数据加载与预处理函数（matrix_list2中的数据已处理好）
# --------------------------------------------------------------
def load_data(file_path):
    """
    按行加载文本文件数据，每行以空格分隔为浮点数列表
    """
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.strip():
                float_values = [float(value) for value in line.split()]
                data.append(float_values)
    return data

def pad_data(data_list):
    """
    使用零填充构造统一大小的 numpy 数组
    """
    num_rows = len(data_list)
    max_length = max(len(row) for row in data_list)
    pad = np.zeros((num_rows, max_length), dtype=np.float32)
    for i, row in enumerate(data_list):
        pad[i, :len(row)] = np.array(row, dtype=np.float32)
    return pad

def convert_data(data):
    """
    对数据中大于0.1的值进行转换（data = 337.5 / data），0值保持不变
    """
    rows, cols = data.shape
    new_data = data.copy()
    for i in range(rows):
        for j in range(cols):
            if new_data[i, j] > 0.1:
                new_data[i, j] = 337.5 / new_data[i, j]
    return new_data

def Find_vacancy_insert(data, l, upscale_factor=4):
    """
    插值填充缺失数据（值为0的位置）
    """
    data_tail = data[:, -l:]
    data_head = data[:, 0:l]
    data_ext = np.concatenate((data_tail, data, data_head), axis=1)
    n_rows = data_ext.shape[0]
    for i in range(n_rows):
        tmp1 = -1
        vacant_number = 0
        for j in range(len(data_ext[i])):
            if data_ext[i, j] == 0 and vacant_number == 0:
                tmp1 = j
                vacant_number += 1
            elif data_ext[i, j] == 0:
                vacant_number += 1
            elif data_ext[i, j] != 0 and (1 <= vacant_number <= l):
                data_ext[i, tmp1:j] = np.nan
                vacant_number = 0
            elif data_ext[i, j] != 0 and vacant_number > l:
                vacant_number = 0
        if 1 <= vacant_number <= l:
            data_ext[i, tmp1:] = np.nan
    for i in range(n_rows):
        row = data_ext[i]
        nan_idx = np.where(np.isnan(row))[0]
        valid_idx = np.where(~np.isnan(row))[0]
        if valid_idx.size > 0 and nan_idx.size > 0:
            f = interp1d(valid_idx, row[valid_idx], kind='linear', fill_value="extrapolate")
            data_ext[i, nan_idx] = f(nan_idx)
    x_dense = np.linspace(0, data_ext.shape[1] - 1, data_ext.shape[1] * upscale_factor)
    data_dense = np.zeros((data_ext.shape[0], len(x_dense)))
    for i in range(data_ext.shape[0]):
        f = interp1d(np.arange(data_ext.shape[1]), data_ext[i], kind='linear')
        data_dense[i] = f(x_dense)
    return data_dense[:, l * upscale_factor: -l * upscale_factor]

# --------------------------------------------------------------
# 工具函数：将2D数据转换为同心圆坐标
# --------------------------------------------------------------
def get_circle_coords(data, max_radius=5):
    """
    将二维矩阵 data 转换为同心圆坐标，返回 x_all, y_all, c_all（c_all 为数据展开后的值）
    参数：
      data: 每行代表一个圆环的采样值；
      max_radius: 图中最大半径，用于确定采样步长
    """
    data_t = data.T
    n_rings = data_t.shape[0]
    radius_step = max_radius / n_rings
    x_list, y_list = [], []
    for i in range(n_rings):
        theta = np.linspace(0, 2 * np.pi, data_t.shape[1], endpoint=False)
        r = i * radius_step
        x_list.append(r * np.cos(theta))
        y_list.append(r * np.sin(theta))
    x_all = np.concatenate(x_list)
    y_all = np.concatenate(y_list)
    c_all = np.concatenate(list(data_t))
    return x_all, y_all, c_all

# --------------------------------------------------------------
# 自定义颜色映射函数
# --------------------------------------------------------------
def _get_custom_cmap():
    """
    构造自定义颜色映射，数据值范围35到50，颜色节点如下：
      35：#000064  
      36：#00007f  
      37：#0000d1  
      38：#002dff  
      39：#0077ff  
      40：#00c6ff  
      41：#00ffd8  
      42：#00ff70  
      43：#00ff00  
      44：#70ff00  
      45：#d8ff00  
      46：#ffce00  
      47：#ff8e00  
      48：#ff4f00  
      49：#ff0f00  
      50：#bf0000
    """
    color_list = [
        (0.0, "#000064"),               # 35 -> normalized 0.0
        (1/15, "#00007f"),              # 36 -> (36-35)/15 ≈ 0.0667
        (2/15, "#0000d1"),              # 37 -> (37-35)/15 ≈ 0.1333
        (3/15, "#002dff"),              # 38 -> (38-35)/15 = 0.2
        (4/15, "#0077ff"),              # 39 -> (39-35)/15 ≈ 0.2667
        (5/15, "#00c6ff"),              # 40 -> (40-35)/15 ≈ 0.3333
        (6/15, "#00ffd8"),              # 41 -> (41-35)/15 = 0.4
        (7/15, "#00ff70"),              # 42 -> (42-35)/15 ≈ 0.4667
        (8/15, "#00ff00"),              # 43 -> (43-35)/15 ≈ 0.5333
        (9/15, "#70ff00"),              # 44 -> (44-35)/15 = 0.6
        (10/15, "#d8ff00"),             # 45 -> (45-35)/15 ≈ 0.6667
        (11/15, "#ffce00"),             # 46 -> (46-35)/15 ≈ 0.7333
        (12/15, "#ff8e00"),             # 47 -> (47-35)/15 = 0.8
        (13/15, "#ff4f00"),             # 48 -> (48-35)/15 ≈ 0.8667
        (14/15, "#ff0f00"),             # 49 -> (49-35)/15 ≈ 0.9333
        (1.0, "#bf0000")                # 50 -> normalized 1.0
    ]
    cmap = LinearSegmentedColormap.from_list("custom_heat", color_list, N=256)
    return cmap

# --------------------------------------------------------------
# 可视化函数——将数据以同心圆格式显示
# --------------------------------------------------------------
def visualize_circle(data, title="", max_radius=5, save_path=None):
    """
    将二维矩阵 data 转换为同心圆图并显示，并可选择保存图像
    """
    x_all, y_all, c_all = get_circle_coords(data, max_radius=max_radius)
    cmap = _get_custom_cmap()  # 使用自定义颜色映射
    
    plt.figure(figsize=(10, 8))
    sc = plt.scatter(x_all, y_all, c=c_all, cmap=cmap)
    sc.set_clim(30, 50)  # 设置colorbar范围

    plt.title(title, fontsize=14)
    plt.axis('off')
    plt.xlim(-max_radius, max_radius)
    plt.ylim(-max_radius, max_radius)

    cbar = plt.colorbar(sc)  # colorbar 会遵循设定的 clim
    
    # 保存图像
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')

    plt.show()




import os
import glob

# 原始数据路径
root_dir = "/Users/txh/Desktop/research/lwk/ok/6.0_after_filtering"
# 目标存储路径
output_root = "/Users/txh/Desktop/research/lwk/ok/6.0_original"

# 遍历所有 .tgl 文件
file_paths = sorted(glob.glob(os.path.join(root_dir, "**", "*.tgl"), recursive=True))

for file_path in file_paths:
    # 加载数据
    data = load_data(file_path)
    padded_data = pad_data(data)
    inserted_data = Find_vacancy_insert(padded_data, 15, upscale_factor=10)  # 增加插值密集度
    
    # 计算对应的相对路径
    relative_path = os.path.relpath(file_path, root_dir)  # 获取相对路径
    new_dir = os.path.join(output_root, os.path.dirname(relative_path))  # 生成新目录路径
    os.makedirs(new_dir, exist_ok=True)  # 递归创建目录（如果不存在）

    # 生成新文件路径
    new_file_name = os.path.basename(file_path).replace(".tgl", "_original.png")  # 修改文件名
    save_path = os.path.join(new_dir, new_file_name)  # 拼接最终存储路径

    # 处理并保存
    visualize_circle(inserted_data, title="test", max_radius=5, save_path=save_path)

    print(f"处理完成: {file_path} -> {save_path}")



## Segmentation

In [26]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import os
import glob

def kmeans_segmentation(image_path, k=3, save_path=None):
    """
    利用 K-means 聚类对图像像素进行分割。
    """
    # 读取图像
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"无法加载图像: {image_path}")
    
    # 转换为 RGB 方便后续处理
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pixel_vals = image_rgb.reshape((-1, 3))
    pixel_vals = np.float32(pixel_vals)

    # K-means 聚类
    kmeans = KMeans(n_clusters=k, n_init=10, max_iter=100, random_state=42)
    labels = kmeans.fit_predict(pixel_vals)
    centers = kmeans.cluster_centers_.astype(np.uint8)

    # 生成分割后的图像
    segmented_data = centers[labels]
    seg_img_rgb = segmented_data.reshape(image_rgb.shape)

    # 转换回 BGR 格式
    seg_img = cv2.cvtColor(seg_img_rgb, cv2.COLOR_RGB2BGR)
    labels_2d = labels.reshape((image.shape[0], image.shape[1]))

    # **保存图像**
    if save_path:
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        cv2.imwrite(save_path, seg_img)

    return labels_2d, centers, seg_img

# **路径定义**
input_root = "/Users/txh/Desktop/research/lwk/ok/6.0_original"  # 原始图像目录
output_root = "/Users/txh/Desktop/kmeans_segment"  # 结果保存目录

# 遍历所有 .png 文件
file_paths = sorted(glob.glob(os.path.join(input_root, "**", "*.png"), recursive=True))

for file_path in file_paths:
    # 计算对应的相对路径
    relative_path = os.path.relpath(file_path, input_root)
    new_dir = os.path.join(output_root, os.path.dirname(relative_path))  # 生成新目录路径
    os.makedirs(new_dir, exist_ok=True)  # 递归创建目录

    # 生成新文件路径
    new_file_name = os.path.basename(file_path).replace(".png", "_kmeans_segment.png")  # 修改文件名
    save_path = os.path.join(new_dir, new_file_name)

    # 处理并保存
    kmeans_segmentation(file_path, k=4, save_path=save_path)

    print(f"处理完成: {file_path} -> {save_path}")


处理完成: /Users/txh/Desktop/research/lwk/ok/6.0_original/1/period1_0001yukaize_Left_2020-8-15_15-03-51_original.png -> /Users/txh/Desktop/kmeans_segment/1/period1_0001yukaize_Left_2020-8-15_15-03-51_original_kmeans_segment.png
处理完成: /Users/txh/Desktop/research/lwk/ok/6.0_original/1/period2_0001yukaize_Left_2022-01-8_14-05-36_original.png -> /Users/txh/Desktop/kmeans_segment/1/period2_0001yukaize_Left_2022-01-8_14-05-36_original_kmeans_segment.png
处理完成: /Users/txh/Desktop/research/lwk/ok/6.0_original/1/period3_0001yukaize_Left_2022-02-19_14-05-29_original.png -> /Users/txh/Desktop/kmeans_segment/1/period3_0001yukaize_Left_2022-02-19_14-05-29_original_kmeans_segment.png
处理完成: /Users/txh/Desktop/research/lwk/ok/6.0_original/1/period4_0001yukaize_Left_2022-06-24_13-58-25_original.png -> /Users/txh/Desktop/kmeans_segment/1/period4_0001yukaize_Left_2022-06-24_13-58-25_original_kmeans_segment.png
处理完成: /Users/txh/Desktop/research/lwk/ok/6.0_original/1/period5_0001yukaize_Left_2022-10-22_14-29-43